<a href="https://colab.research.google.com/github/Mahedi-Hasan-Anik/Thesis_materials/blob/main/Talha_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from glob import glob
import os
!pip install mne
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
#read all file
all_files_path=glob('/content/drive/MyDrive/dataverse/*.edf')
print(len(all_files_path))

28


In [22]:
all_files_path

['/content/drive/MyDrive/dataverse/h01.edf',
 '/content/drive/MyDrive/dataverse/h04.edf',
 '/content/drive/MyDrive/dataverse/h05.edf',
 '/content/drive/MyDrive/dataverse/h02.edf',
 '/content/drive/MyDrive/dataverse/h03.edf',
 '/content/drive/MyDrive/dataverse/h12.edf',
 '/content/drive/MyDrive/dataverse/h09.edf',
 '/content/drive/MyDrive/dataverse/h11.edf',
 '/content/drive/MyDrive/dataverse/h13.edf',
 '/content/drive/MyDrive/dataverse/h07.edf',
 '/content/drive/MyDrive/dataverse/h08.edf',
 '/content/drive/MyDrive/dataverse/h10.edf',
 '/content/drive/MyDrive/dataverse/h06.edf',
 '/content/drive/MyDrive/dataverse/s05.edf',
 '/content/drive/MyDrive/dataverse/s03.edf',
 '/content/drive/MyDrive/dataverse/s01.edf',
 '/content/drive/MyDrive/dataverse/h14.edf',
 '/content/drive/MyDrive/dataverse/s06.edf',
 '/content/drive/MyDrive/dataverse/s04.edf',
 '/content/drive/MyDrive/dataverse/s02.edf',
 '/content/drive/MyDrive/dataverse/s10.edf',
 '/content/drive/MyDrive/dataverse/s07.edf',
 '/content

In [ ]:
# healthy_file_path=[i for i in all_files_path if  'h' in i.split('/')[1]]
healthy_file_path = [i for i in all_files_path if 'h' in i.split('/content/drive/MyDrive/dataverse/')[1]]
patient_file_path=[i for i in all_files_path if  's' in i.split('/content/drive/MyDrive/dataverse/')[1]]
patient_file_path

In [37]:
datay=mne.io.read_raw_edf(healthy_file_path[0],preload=True)
datay

Extracting EDF parameters from /content/drive/MyDrive/dataverse/h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...


Measurement date,"June 23, 2003 16:14:37 GMT"
Experimenter,Unknown
Participant,DJZ
Digitized points,Not available
Good channels,19 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [34]:
def read_data(file_path):
  datax=mne.io.read_raw_edf(file_path,preload=True)
  datax.set_eeg_reference()
  datax.filter(l_freq=1,h_freq=40)
  epochs = mne.make_fixed_length_epochs(datax, duration=60)
  epochs = epochs.get_data()
  return epochs # trials, channel,length


In [ ]:
data = read_data(healthy_file_path[0])

In [38]:
data.shape

(15, 19, 15000)

In [ ]:
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]
# control_epochs_array

In [40]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))

14 14


In [44]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
print(len(data_list),len(label_list))

28 28


In [46]:
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]


In [47]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(groups_list)
print(data_array.shape,label_array.shape,group_array.shape)

(472, 19, 15000) (472,) (472,)


In [48]:
from scipy import stats
def mean(data):
    return np.mean(data,axis=-1)

def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
        return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)


def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)


def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [49]:
from tqdm import tqdm_notebook
features=[]
for data in tqdm_notebook(data_array):
    features.append(concatenate_features(data))
features=np.array(features)
features.shape

<ipython-input-49-69f2cdc0fdb1>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data in tqdm_notebook(data_array):


  0%|          | 0/472 [00:00<?, ?it/s]

(472, 247)